Сегодня мы поговорим о том, как доставать из сырого текста связи между объектами

## предустановка всего необходимого

In [ ]:
# поставим Spacy, оттуда будем загружать модели NER
# ! pip3 install -U spacy==2.1.0

!pip3 install spacy
!python3 -m spacy download en
! pip3 install tqdm

In [1]:
# сделаем еще импорты нужных библиотек

from tqdm.notebook import tqdm # визуализация прогресс-баров

import re # регулярки
from spacy.lang.en import English

# для датафреймов
import pandas as pd
import numpy as np

# визуализации
%matplotlib inline

# импортируем Spacy и нужные части оттуда
import spacy
from spacy import displacy


nlp = spacy.load("en_core_web_sm") # загрузили обученную ML-модель



## Шаг 1: из сырого текста вытащим упоминание сущностей
*(такая задача называется Named Entity Recognition  или NER)*

In [98]:
# text = "When Sebastian Thrun started working on self-driving cars at Google in 2007,\
#  few people outside of the company took him seriously."

text = "I ate two bars of 'Mars', then 'Bounty' and Milky Way"
# print(type(text))

doc = nlp(text) # обрабатываем текст

# print(type(doc))

displacy.render(doc, style = "ent", jupyter = True) # style = "dep" for dependencies

doc.ents

(two,)

In [20]:
doc.ents[-1]

2007

[здесь подробнее про  параметры визуализации](https://spacy.io/usage/visualizers)

In [ ]:
# посмотрим на список сущностей в документе
doc.ents

## экспериментальная часть: попробуем на большом датасете

*(давайте его подготовим)*

In [ ]:
# !unzip elonmusk.csv.zip # запустите ячейку если загружаете файл в заархивированном виде

In [40]:
# откроем датафрейм

df = pd.read_csv("/Users/nstsj/python_for_CL/data/Twitter- Oscars(2).csv", encoding="latin-1")

In [46]:
df.columns=df.iloc[0]

# df.columns

df.head(3)

,Date,Screen Name,Full Name,Tweet Text,Tweet ID,App,Followers,Follows,Retweets,Favorites,Verfied,User Since,Location,Bio,Profile Image,Google Maps
0,Date,Screen Name,Full Name,Tweet Text,Tweet ID,App,Followers,Follows,Retweets,Favorites,Verfied,User Since,Location,Bio,Profile Image,Google Maps
1,2/27/2017 23:48:13,@caileonn,???i?i? ?eo??,"RT @deray: How Far I'll Go, Moana. feat. Auli'...",836452929183731712,Twitter for iPhone,397,295,6777,0,No,8/16/2009,West Coast,{ Instagram: @caileonn },View,NaN
2,2/27/2017 23:48:13,@luisdemdiaz,Luis Diaz,RT @cosmos212: Será la foto de la década?? #os...,836452930601484288,Twitter Web Client,67,119,77,0,No,2/23/2016,"Culiacán, Sinaloa",25 años.,View,NaN


In [125]:
# df cleaning
df.dropna(inplace=True)
df_sma = df[["Tweet Text","Screen Name"]]\
    .rename(columns={"Tweet Text":"text", "Screen Name":"username"}) # посмотрим на содержимое колонки с твитами


In [146]:
clean = df_sm.sample(n=100, random_state=3)
clean = clean.reset_index().drop("index", axis=1)
clean

,text,username
0,RT @SeriesBrasil: Dakota Johnson na after part...,@sheszquad
1,RT @itzsmiley_: ???? ??????? ??????? ??????? ?...,@Gunasekarkpr
2,RT @mydigi: Edisi #Oscars RT kalau faham! ?? #...,@syafeeqz95
3,RT @Nangdee: ???????????????????? ????????????...,@NAEME_TK
4,RT @9GAG: Oh my god! #Oscars https://t.co/gDCd...,@LianMAniez
...,...,...
95,RT @colinjones: Here's Viola Davis' full #Osca...,@Bhudda__Rex
96,RT @dylanohollands: 2017: the year of viola da...,@jennyluvsme
97,RT @1910_seba: ??? ??????????!?!??!??? ¿??¿ ?!...,@dianauldashova
98,Congratulations #ViolaDavis #Oscars You're ama...,@Desinskeycrazy


In [156]:
oscars = clean.text.to_list()

len(oscars)

100

In [160]:
 # для твитов выбранного диапазона (его можно менять)
for x in oscars[87:91]:
    text = re.sub("RT\s@\w+:","",x)
    doc = nlp(text) # обрабатываем текст spacy-функцией
    displacy.render(doc, style = "ent",jupyter =True) # отображаем такие твиты 
    print('\nEntities found:',doc.ents) # покажите сами сущности



Entities found: (Congrats, Steve Harvey)



Entities found: (9,)



Entities found: ()



Entities found: (#,)


In [140]:
# as a func

def get_ents(text):
    
        text = re.sub("RT\s@\w+:","",text) # убираем мусор
        doc = nlp(text) # обрабатываем текст spacy-функцией
        res = list(doc.ents)
        
        return res

In [169]:
# clean["entities"] = clean.text.apply(lambda x: get_ents(x))
clean[54:59]

,text,username,entities
54,RT @beynobrasil: Beyoncé atualizou seu site co...,@yezzy_zlut,[(Beyoncé)]
55,RT @OddNaari: #Moonlight has become the first ...,@Say_Ouate,"[(first), (#, re, #)]"
56,#Oscars https://t.co/kYIY64e15B,@casanord10,[]
57,RT @_getadvice: ??????? ????????? «??????» ???...,@lery178,"[(#, Oscars, #, Lalaland)]"
58,RT @mrskin: Going for the Gold': Oscar Nods S...,@NilzLuet,"[(Gold), (Oscar, Nods, Show, Their, Bods)]"


# Как еще можно извлекать информацию?
- не через сущности, а через POS-тэги,([так](https://github.com/nstsj/Knowledge_Engineering/blob/master/class1/KE_%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D1%8F1.pptx),  [так](https://github.com/nstsj/Knowledge_Engineering/blob/master/class2/KE_%D1%81%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%802_Knowledge_Graphs.ipynb))

- из русскоязычных текстов, при помощи [natasha](https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/NER.ipynb)

- с помощью [этой библиотеки](https://github.com/fhamborg/Giveme5W1H) (работает с json-файлами)
